In [1]:
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

import torch, torchvision
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    print('Using GPU.')
    print('Memory Allocated: {}'.format(torch.cuda.memory_allocated()))
    print('Max Memory Allocated: {}'.format(torch.cuda.max_memory_allocated()))
    print('Memory Reserved: {}'.format(torch.cuda.memory_reserved()))
else:
    print('No useable GPUs found.  Using CPU.')

# Read recipe inputs
data_input = dataiku.Folder("loYoKVsM")
input_path = data_input.get_path()
model_output = dataiku.Folder("UrfVTR9K")
output_path = model_output.get_path()

Using GPU.
Memory Allocated: 0
Max Memory Allocated: 0
Memory Reserved: 0


In [0]:
from shutil import copyfile

from PIL import Image
import os
import json

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import os
import json


# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

json_path = ''
for f in os.listdir(input_path):
    if f.endswith('.json'):
        json_path = os.path.join(input_path, f)
        break


for f in os.listdir(input_path):
    if f.endswith('.json'):
        break
with open(os.path.join(input_path, f), 'r') as file:
    data = json.load(file)
classes = []
for thing in data['categories']:
    classes.append(thing['name'])


backbone = None#config['backbone']
ims_per_batch = 4#int(config['ims_per_batch'])
epochs = 10000#int(config['epochs'])
resume = False#config['resume']
base_lr = 0.0001#config['base_lr']
num_checkpoints = 5#config['num_checkpoints']
step = True#config['step']

my_dict = {'backbone': backbone,
           'ims_per_batch': ims_per_batch,
           'epochs': epochs,
           'resume': resume,
           'base_lr': base_lr,
           'num_checkpoints': num_checkpoints,
           'step': step}






with open(os.path.join(output_path, 'config.json'), 'w') as outfile:
    json.dump(my_dict, outfile)

register_coco_instances("train_data", {}, json_path, input_path)
#register_coco_instances("CarDamage_val", {}, os.path.join(car_Damage_Data_path, 'COCO_train_annos.json'), input_path)
backbone = 'mask_rcnn_R_50_C4_1x.yaml'
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/" + backbone))
cfg.DATASETS.TRAIN = ("train_data",)
cfg.DATASETS.TEST = ("train_data",)
cfg.MODEL.WEIGHTS = os.path.join(output_path, 'model_final.pth')
cfg.SOLVER.IMS_PER_BATCH = ims_per_batch
cfg.SOLVER.BASE_LR = base_lr
cfg.SOLVER.MAX_ITER = epochs
#cfg.BATCHSIZE = 4
cfg.SOLVER.CHECKPOINT_PERIOD = epochs//num_checkpoints
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
if not step:
    cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.OUTPUT_DIR = output_path
cfg.MODEL.DEVICE = device

trainer = DefaultTrainer(cfg)
if not resume:
    for f in os.listdir(output_path):
        os.remove(os.path.join(output_path, f))
trainer.resume_or_load(resume=resume)
trainer.train()

In [0]:
from PIL import Image
import os
import json

cwd = os.getcwd()
os.mkdir('my_images')
os.mkdir('my_model')
print('cwd: {}'.format(cwd))
print(os.listdir())


for img in [item["fullPath"] for item in data_input.get_path_details()["children"]]:
    with data_input.get_download_stream(path=img) as stream:
        if img.endswith('.jpg'):
            im = Image.open(stream)
            im.save(cwd + '/my_images' + img)
        elif img.endswith('.json'):
            string = stream.read().decode('utf-8')
            json_obj = json.loads(string)
            with open(cwd + '/my_images' + img, 'w') as f:
                json.dump(json_obj, f)
        else:
            print('NON-FATAL ERROR: Not an image/json file, file type not supported.')


'''for img in [item["fullPath"] for item in model_input.get_path_details()["children"]]:
    with model_input.get_download_stream(path=img) as stream:
        string = stream.read()'''

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import os
import json


# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

input_path = 'my_images'
output_path = 'my_model'

json_path = ''
for f in os.listdir(input_path):
    if f.endswith('.json'):
        json_path = os.path.join(input_path, f)
        break


for f in os.listdir(input_path):
    if f.endswith('.json'):
        break
with open(os.path.join(input_path, f), 'r') as file:
    data = json.load(file)
classes = []
for thing in data['categories']:
    classes.append(thing['name'])


backbone = None#config['backbone']
ims_per_batch = 16#int(config['ims_per_batch'])
epochs = 10000#int(config['epochs'])
resume = False#config['resume']
base_lr = 0.0001#config['base_lr']
num_checkpoints = 5#config['num_checkpoints']
step = True#config['step']

my_dict = {'backbone': backbone,
           'ims_per_batch': ims_per_batch,
           'epochs': epochs,
           'resume': resume,
           'base_lr': base_lr,
           'num_checkpoints': num_checkpoints,
           'step': step}






with open(os.path.join(output_path, 'config.json'), 'w') as outfile:
    json.dump(my_dict, outfile)

register_coco_instances("train_data", {}, json_path, input_path)
#register_coco_instances("CarDamage_val", {}, os.path.join(car_Damage_Data_path, 'COCO_train_annos.json'), input_path)
if type(backbone)!=str:
    print('wuht:')
    print(backbone)
    backbone = 'mask_rcnn_R_50_C4_1x.yaml'
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/" + backbone))
cfg.DATASETS.TRAIN = ("train_data",)
cfg.DATASETS.TEST = ("train_data",)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/" + backbone)
cfg.SOLVER.IMS_PER_BATCH = ims_per_batch
cfg.INPUT.MIN_SIZE_TRAIN = (400,)
cfg.INPUT.MIN_SIZE_TEST = (400,)
cfg.SOLVER.BASE_LR = base_lr
cfg.SOLVER.MAX_ITER = epochs
#cfg.BATCHSIZE = 4
cfg.SOLVER.CHECKPOINT_PERIOD = epochs//num_checkpoints
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
if not step:
    cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.OUTPUT_DIR = output_path
cfg.MODEL.DEVICE = device

trainer = DefaultTrainer(cfg)
if not resume:
    for f in os.listdir(output_path):
        os.remove(os.path.join(output_path, f))
trainer.resume_or_load(resume=resume)
trainer.train()

'''
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import cv2

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer

import os
import json
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode
from detectron2.data import MetadataCatalog

with open(model_path + '/last_checkpoint', 'r') as f:
    last_model = f.readline()

with open(os.path.join(model_path, 'config.json'), 'r') as f:
    config_data = json.load(f)
backbone = config_data['backbone']

for f in os.listdir(data_path):
    if f.endswith('.json'):
        break
with open(os.path.join(data_path, f), 'r') as file:
    data = json.load(file)
classes = []
for thing in data['categories']:
    classes.append(thing['name'])


MetadataCatalog.get('CarDamage_damage_type_metadata').set(thing_classes=classes)
metadata = MetadataCatalog.get('CarDamage_damage_type_metadata')


cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(backbone))
cfg.MODEL.WEIGHTS = os.path.join(model_path, last_model)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
cfg.MODEL.DEVICE = device
cfg.OUTPUT_DIR = output_path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold
predictor = DefaultPredictor(cfg)

# get the frame width and height
#output.write_json('dummy-file.txt', '{}')


video_path =  input_path + config['video']
for file in os.listdir(input_path):
    if not file.endswith('.json'):
        video_path = os.path.join(input_path, file)

        cap = cv2.VideoCapture(video_path)
        if (cap.isOpened() == False):
            print('Error while trying to read {}. Please check path again.'.format(file))

        threshold = 0.90

        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        output_name =  output_path + "/" + file.split('.')[0] + ".mp4"
        # get fps (try and except for different cv2 versions)
        try:
            fps = cap.get(cv2.cv.CV_CAP_PROP_FPS)
        except:
            fps = cap.get(cv2.CAP_PROP_FPS)
        # define codec and create VideoWriter object
        out = cv2.VideoWriter(output_name,
                          cv2.VideoWriter_fourcc(*'mp4v'), fps,
                          (frame_width, frame_height))
        ret = True
        while ret:
            # capture each frame of the video
            ret, im = cap.read()
            if ret == True:
                # keep a copy of the original image for OpenCV functions and applying masks
                if image_background:
                    orig_image = image.copy()
                else:
                    orig_image = np.zeros_like(image)
                outputs = predictor(im)
                v = Visualizer(im[:, :, ::-1],
                               metadata=metadata,
                               scale=1,
                               instance_mode=ColorMode.IMAGE   # remove the colors of unsegmented pixels
                )
                v = v.draw_instance_predictions(outputs["instances"].to(device))
                new_im = v.get_image()[:, :, ::-1]
                out.write(new_im)
        with video_output.get_writer(file) as w:
            w.write()


            '''




#os.remove(os.path.join(output_path, 'dummy-file.txt'))